# Road Following Live (with camera)

Load the optimized model (created with the [`optimize_model.ipynb` notebook](./optimize_model.ipynb)) executing the cell below

In [1]:
import torch
from torch2trt import TRTModule

model_trt = TRTModule()
model_trt.load_state_dict(torch.load('model_08-06_trt.pth'))

<All keys matched successfully>

Create the racecar class

In [2]:
from jetracer.nvidia_racecar import NvidiaRacecar

car = NvidiaRacecar()

Create the camera class.

In [3]:
from jetcam.csi_camera import CSICamera

camera = CSICamera(width=224, height=224, capture_fps=65)

RuntimeError: Could not initialize camera.  Please see error trace.

Finally, execute the cell below to make the racecar move forward, steering the racecar based on the x value of the apex.

Here are some tips,

* If the car wobbles left and right,  lower the steering gain
* If the car misses turns,  raise the steering gain
* If the car tends right, make the steering bias more negative (in small increments like -0.05)
* If the car tends left, make the steering bias more postive (in small increments +0.05)

In [ ]:
from utils import preprocess
import numpy as np

STEERING_GAIN = 1
STEERING_BIAS = -0.20

car.throttle = 0

while True:
    image = camera.read()
    image = preprocess(image).half()
    output = model_trt(image).detach().cpu().numpy().flatten()
    x = float(output[0])
    car.steering = x * STEERING_GAIN + STEERING_BIAS

In [ ]:
car.throttle = 0.15